<a href="https://colab.research.google.com/github/2303A52114/GENERATIVE-AI/blob/main/ASSIGNMENT_6_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load dataset
file_path = "Housing.csv"
df = pd.read_csv(file_path)

# Identify categorical and numerical columns
categorical_columns = ['mainroad', 'guestroom', 'basement', 'hotwaterheating',
                       'airconditioning', 'prefarea', 'furnishingstatus']
numerical_columns = [col for col in df.columns if col not in categorical_columns + ['price']]

# One-hot encode categorical variables
df = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

# Handle skewness in target variable
df['price'] = np.log1p(df['price'])

# Separate features and target variable
X = df.drop(columns=['price'])
y = df['price']

# Normalize numerical features
scaler = StandardScaler()
X[numerical_columns] = scaler.fit_transform(X[numerical_columns])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the improved ANN model
model = Sequential([
    Dense(64, activation='relu', kernel_regularizer=l2(0.001), input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.3),
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.2),
    Dense(1, activation='linear')
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='mean_squared_error',
              metrics=['mae'])

# Early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
model.fit(X_train, y_train, epochs=200, batch_size=32,
          validation_data=(X_test, y_test), verbose=1, callbacks=[early_stopping])

# Evaluate the model
train_loss, train_mae = model.evaluate(X_train, y_train, verbose=1)
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=1)
print(f'Training MAE: {train_mae}, Testing MAE: {test_mae}')

# Save the model
model.save("/mnt/data/housing_price_model_optimized.h5")
print("Optimized model saved as 'housing_price_model_optimized.h5'.")

# Load and use the saved model
loaded_model = tf.keras.models.load_model("/mnt/data/housing_price_model_optimized.h5")
predictions = np.expm1(loaded_model.predict(X_test[:5]))

# Display sample predictions
print("Sample Predictions:")
print(predictions)

Epoch 1/200


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


14/14 ━━━━━━━━━━━━━━━━━━━━ 5s 74ms/step - loss: 212.9240 - mae: 14.5687 - val_loss: 165.2116 - val_mae: 12.8305
Epoch 2/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 134.5067 - mae: 11.4570 - val_loss: 46.6679 - val_mae: 6.5340
Epoch 3/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 30.5498 - mae: 4.7719 - val_loss: 16.0218 - val_mae: 3.1756
Epoch 4/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 19.8245 - mae: 3.4501 - val_loss: 10.2346 - val_mae: 2.6032
Epoch 5/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 13.0721 - mae: 2.8985 - val_loss: 7.7606 - val_mae: 2.2563
Epoch 6/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 10.9313 - mae: 2.6698 - val_loss: 6.8265 - val_mae: 2.1417
Epoch 7/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 10.4020 - mae: 2.6066 - val_loss: 6.8695 - val_mae: 2.1298
Epoch 8/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 10.0513 - mae: 2.4938 - val_loss: 6.3461 - val_mae: 2.0502
Epoch 9/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss

Training MAE: 0.5689617395401001, Testing MAE: 0.8213415741920471
Optimized model saved as 'housing_price_model_optimized.h5'.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
Sample Predictions:
[[ 182290.5]
 [2115203.2]
 [4578314. ]
 [3066397. ]
 [2163737.2]]
